In [1]:
import numpy as np
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/Users/louisalu/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/louisalu/Documents/04WSP/venv/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#!pip install bokeh==2.4.2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df_price = (
    pd.read_csv('dowj_price_indicators.csv', index_col=0)
    .rename(columns={'Close*':'Close'})
    [['Date', 'Open', 'High', 'Low', 'Close']]
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [4]:
df_price

,Open,High,Low,Close
Date,,,,
2008-09-17,11056.58,11057.31,10595.90,10609.66
2008-09-18,10609.01,11076.44,10459.44,11019.69
2008-09-19,11027.51,11483.05,11026.70,11388.44
2008-09-22,11394.42,11394.58,10992.20,11015.69
2008-09-23,11015.69,11143.21,10833.94,10854.17
...,...,...,...,...
2016-06-27,17355.21,17355.21,17063.08,17140.24
2016-06-28,17190.51,17409.72,17190.51,17409.72
2016-06-29,17456.02,17704.51,17456.02,17694.68


In [5]:
df_predictions = (
    pd.read_csv('prediction_output.csv', index_col=0)
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [6]:
df_predictions

,iterate_prediction,grid_prediction,random_prediction
Date,,,
2014-03-05,-0.000488,0.000614,-0.002610
2014-03-06,0.004805,0.002387,-0.000320
2014-03-07,0.001993,0.002387,-0.000598
2014-03-10,0.003717,0.001717,-0.001000
2014-03-11,-0.001528,0.001017,-0.000465
...,...,...,...
2016-06-27,0.035532,0.005833,0.004592
2016-06-28,0.022988,0.012130,0.011819
2016-06-29,0.026804,0.005521,0.003258


In [7]:
# Predictions includes only testing data so the resulting df has only that as well
df_price_and_predictions = df_predictions.join(df_price)
df_price_and_predictions

,iterate_prediction,grid_prediction,random_prediction,Open,High,Low,Close
Date,,,,,,,
2014-03-05,-0.000488,0.000614,-0.002610,16395.88,16406.55,16343.96,16360.18
2014-03-06,0.004805,0.002387,-0.000320,16360.56,16450.17,16360.56,16421.89
2014-03-07,0.001993,0.002387,-0.000598,16424.53,16505.70,16398.86,16452.72
2014-03-10,0.003717,0.001717,-0.001000,16453.10,16453.10,16334.20,16418.68
2014-03-11,-0.001528,0.001017,-0.000465,16419.39,16460.33,16325.17,16351.25
...,...,...,...,...,...,...,...
2016-06-27,0.035532,0.005833,0.004592,17355.21,17355.21,17063.08,17140.24
2016-06-28,0.022988,0.012130,0.011819,17190.51,17409.72,17190.51,17409.72
2016-06-29,0.026804,0.005521,0.003258,17456.02,17704.51,17456.02,17694.68


In [8]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat1(Strategy):
    """
    Opens a long position whenever 2 or more prediction values
    are positive. Close it otherwise.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        if (ps1 + ps2 + ps3) >= 2:
            self.buy()
        else:
            self.position.close()


In [9]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat1, cash=100_000_000)#, exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2014-03-05 00:00:00
End                       2016-07-01 00:00:00
Duration                    849 days 00:00:00
Exposure Time [%]                   73.469388
Equity Final [$]                 112374609.33
Equity Peak [$]                  113294203.33
Return [%]                          12.374609
Buy & Hold Return [%]                9.713768
Return (Ann.) [%]                    5.127162
Volatility (Ann.) [%]               12.056266
Sharpe Ratio                         0.425269
Sortino Ratio                        0.639735
Calmar Ratio                         0.530425
Max. Drawdown [%]                   -9.666147
Avg. Drawdown [%]                   -2.331028
Max. Drawdown Duration      164 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                  108
Win Rate [%]                        58.333333
Best Trade [%]                       3.587229
Worst Trade [%]                     -5.049272
Avg. Trade [%]                    

Row(id='1555', ...)

In [10]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat2(Strategy):
    """
    Open a long position whenever the 3 prediction values are positive.
    Open a short position whenever the 3 prediction values are negative.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        # Negative signals
        ns1 = int(self.p1[-1] < 0)
        ns2 = int(self.p2[-1] < 0)
        ns3 = int(self.p3[-1] < 0)

        if (ps1 + ps2 + ps3) == 3:
            self.buy()
        elif (ns1 + ns2 + ns3) == 3:
            self.sell()


In [11]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat2, cash=100_000_000, exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2014-03-05 00:00:00
End                       2016-07-01 00:00:00
Duration                    849 days 00:00:00
Exposure Time [%]                   97.959184
Equity Final [$]                 125937480.41
Equity Peak [$]                  128048092.25
Return [%]                           25.93748
Buy & Hold Return [%]                9.713768
Return (Ann.) [%]                   10.388434
Volatility (Ann.) [%]                15.22537
Sharpe Ratio                         0.682311
Sortino Ratio                        1.111478
Calmar Ratio                         0.884584
Max. Drawdown [%]                  -11.743865
Avg. Drawdown [%]                   -2.521663
Max. Drawdown Duration      251 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                  247
Win Rate [%]                        54.251012
Best Trade [%]                       3.043256
Worst Trade [%]                     -4.640792
Avg. Trade [%]                    

Row(id='2512', ...)